In [9]:
using Statistics
using Flux
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated
using CuArrays

In [10]:
imgs = Flux.Data.MNIST.images()
labels = Flux.Data.MNIST.labels();

┌ Info: Downloading MNIST dataset
└ @ Flux.Data.MNIST /home/dreuter/.julia/packages/Flux/qXNjB/src/data/mnist.jl:24
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   469  100   469    0     0    309      0  0:00:01  0:00:01 --:--:--   309
100 9680k  100 9680k    0     0  2034k      0  0:00:04  0:00:04 --:--:-- 4051k
┌ Info: Downloading MNIST dataset
└ @ Flux.Data.MNIST /home/dreuter/.julia/packages/Flux/qXNjB/src/data/mnist.jl:24
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   469  100   469    0     0    827      0 --:--:-- --:--:-- --:--:--   825
100 28881  100 28881    0     0  22705      0  0:00:01  0:00:01 --:--:-- 22705
┌ Info: Downloading MNIST dataset
└ @ Flux.Data.MNIST /home/dreuter/.julia/packages/Flux/qXNjB/src/data/mnist.jl:24
  % Total    % Recei

In [11]:
imgs[27454] # pick a number 1-60000

In [12]:
labels[27454]

7

In [13]:
## Boring Preprocessing
X = hcat(float.(reshape.(imgs, :))...) #stack all the images
Y = onehotbatch(labels, 0:9) # just a common way to encode categorical variables

10×60000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 false   true  false  false  false  …  false  false  false  false  false
 false  false  false   true  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false   true  false  false  false
 false  false   true  false  false     false  false  false  false  false
  true  false  false  false  false  …  false  false   true  false  false
 false  false  false  false  false     false  false  false   true  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false      true  false  false  false   true
 false  false  false  false   true     false  false  false  false  false

In [14]:
# Our model, just like before, chaining dense layers
# Go from 28^2 dimensional space (images are 28x28) to 10 dimensional space (labels are 0-9)

m = Chain(
  Dense(28^2, 32, relu),
  Dense(32, 10),
  softmax)

# softmax just converts output to probability distribution

Chain(Dense(784, 32, NNlib.relu), Dense(32, 10), NNlib.softmax)

In [15]:
loss(x, y) = crossentropy(m(x), y) 
opt = ADAM(); # popular stochastic gradient descent variant

accuracy(x, y) = mean(onecold(m(x)) .== onecold(y)) # cute way to find average of correct guesses

dataset = repeated((X,Y),200) # repeat the data set 200 times, as opposed to @epochs 200 ...
evalcb = () -> @show(loss(X, Y)) # callback to show loss

#3 (generic function with 1 method)

In [16]:
Flux.train!(loss, params(m), dataset, opt, cb = throttle(evalcb, 10)); #took me ~5 minutes to train on CPU

loss(X, Y) = 2.3161428f0 (tracked)
loss(X, Y) = 1.379861f0 (tracked)
loss(X, Y) = 0.81483394f0 (tracked)
loss(X, Y) = 0.55479205f0 (tracked)
loss(X, Y) = 0.44630384f0 (tracked)
loss(X, Y) = 0.3894398f0 (tracked)
loss(X, Y) = 0.353773f0 (tracked)
loss(X, Y) = 0.32890916f0 (tracked)
loss(X, Y) = 0.30841336f0 (tracked)
loss(X, Y) = 0.29168874f0 (tracked)
loss(X, Y) = 0.277501f0 (tracked)


In [17]:
Flux.Data.MNIST.images(:test)[5287] # give me a number 1-10000

In [19]:
# Same preprocessing
test_X = hcat(float.(reshape.(Flux.Data.MNIST.images(:test), :))...)
test_Y = onehotbatch(Flux.Data.MNIST.labels(:test), 0:9);

m(test_X[:,5287]) # Note the 7th index ( corresponding to the digit 6 ) is nearly 1

Tracked 10-element Array{Float32,1}:
 8.021768f-6    
 6.298194f-8    
 9.481713f-5    
 8.56824f-6     
 0.0001303093f0 
 0.00010362802f0
 0.99951935f0   
 1.425259f-6    
 8.97845f-5     
 4.391449f-5    

In [20]:
#decode
onecold(m(test_X[:,5287])) - 1 #minus 1 since we start from 0, but indexing in Julia starts at 1

6

In [21]:
# Training set accuracy
accuracy(X, Y)

0.9248833333333333

In [22]:
# Test set accuracy
accuracy(test_X, test_Y)

0.9247